In [2]:
# Установка необходимых библиотек
!pip install pandas numpy matplotlib seaborn scikit-learn xgboost lightgbm shap openpyxl

In [3]:
import os
# Импорт библиотек
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Машинное обучение
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score
import shap

# Настройка отображения
plt.style.use('seaborn-v0_8')
pd.set_option('display.max_columns', None)

In [1]:
# === КОД ДЛЯ ПРОГНОЗИРОВАНИЯ НА ОСНОВЕ ОБУЧЕННЫХ МОДЕЛЕЙ ===
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import os
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
import pickle

# Настройка отображения
plt.style.use('seaborn-v0_8')
pd.set_option('display.max_columns', None)

# Загружаем функции предобработки (те же, что использовались при обучении)
def calculate_phenological_features(df):
    df = df.sort_values(['Год', 'Месяц', 'День']).reset_index(drop=True)

    # === Интеграл температур выше 5°C ===
    df['Темп_выше_5'] = np.where(df['Среднесуточная температура'] > 5, df['Среднесуточная температура'] - 5, 0)
    df['Накоп_темп_5'] = df.groupby('Год')['Темп_выше_5'].cumsum()

    # === Интеграл температур выше 10°C ===
    df['Темп_выше_10'] = np.where(df['Среднесуточная температура'] > 10, df['Среднесуточная температура'] - 10, 0)
    df['Накоп_темп_10'] = df.groupby('Год')['Темп_выше_10'].cumsum()

    # === Количество дней с положительной температурой подряд ===
    df['Темп_полож'] = (df['Среднесуточная температура'] > 0).astype(int)
    df['Дни_подряд_полож_температура'] = df.groupby('Год')['Темп_полож'].apply(
        lambda x: x * (x.groupby((x != 1).cumsum()).cumcount() + 1)
    ).values

    # === Осадки за последние 3 и 7 дней ===
    df['Осадки_за_3дня'] = df.groupby('Год')['Осадки'].rolling(window=3, min_periods=1).sum().reset_index(level=0, drop=True)
    df['Осадки_за_7дней'] = df.groupby('Год')['Осадки'].rolling(window=7, min_periods=1).sum().reset_index(level=0, drop=True)

    return df

def add_lag_features(df):
    lags = [1, 2, 3, 5, 7]
    for lag in lags:
        for col in ['Среднесуточная температура', 'Осадки', 'Сред_скорость_ветра', 'Отн_влажность_воздуха, %']:
            if col in df.columns:
                df[f'{col}_лаг_{lag}'] = df.groupby('Год')[col].shift(lag)
    return df

def add_azimuth_vectors(df):
    if 'Азимут' in df.columns:
        df['Азимут_rad'] = np.deg2rad(df['Азимут'])
        df['Азимут_cos'] = np.cos(df['Азимут_rad'])
        df['Азимут_sin'] = np.sin(df['Азимут_rad'])

        # Вектор скорости ветра
        df['Ветер_вектор_X'] = df['Сред_скорость_ветра'] * df['Азимут_cos']
        df['Ветер_вектор_Y'] = df['Сред_скорость_ветра'] * df['Азимут_sin']
    return df

def add_rolling_and_cumulative_features(df):
    for col in ['Среднесуточная температура', 'Осадки', 'Сред_скорость_ветра', 'Отн_влажность_воздуха, %']:
        if col in df.columns:
            # Скользящие средние
            df[f'{col}_скользящее_3'] = df.groupby('Год')[col].rolling(window=3, min_periods=1).mean().reset_index(level=0, drop=True)
            df[f'{col}_скользящее_7'] = df.groupby('Год')[col].rolling(window=7, min_periods=1).mean().reset_index(level=0, drop=True)
            # Скользящие суммы
            df[f'{col}_сумма_3'] = df.groupby('Год')[col].rolling(window=3, min_periods=1).sum().reset_index(level=0, drop=True)
            df[f'{col}_сумма_7'] = df.groupby('Год')[col].rolling(window=7, min_periods=1).sum().reset_index(level=0, drop=True)
    return df

def preprocess_features(df):
    """Функция предобработки признаков (такая же как при обучении)"""
    df_processed = df.copy()

    # Удаляем дублирующий столбец температуры
    if 'Среднесуточная температура' in df_processed.columns:
        temp_cols = [col for col in df_processed.columns if 'Среднесуточная температура' in str(col)]
        if len(temp_cols) > 1:
            df_processed = df_processed.drop(columns=temp_cols[1:])

    # Обработка даты
    if 'Дата' in df_processed.columns:
        df_processed['Дата'] = pd.to_datetime(df_processed['Дата'], dayfirst=True, errors='coerce')
    else:
        # Создаем дату из года, месяца, дня
        df_processed['Дата'] = pd.to_datetime(
            df_processed[['Год', 'Месяц', 'День']].astype(str).agg('-'.join, axis=1),
            errors='coerce'
        )

    # Преобразование румба в азимут
    def rum_to_azimuth(rum):
        rum_dict = {
            'С': 0, 'СВ': 45, 'В': 90, 'ЮВ': 135,
            'Ю': 180, 'ЮЗ': 225, 'З': 270, 'СЗ': 315,
            'ВЮВ': 112.5, 'ЮЮВ': 157.5, 'ЗЮЗ': 247.5, 'СЗС': 337.5
        }
        return rum_dict.get(rum, np.nan)

    if 'Румб' in df_processed.columns:
        df_processed['Азимут'] = df_processed['Румб'].apply(rum_to_azimuth)
        df_processed = df_processed.drop(columns=['Румб'])

    # Кодирование категориальных переменных
    categorical_cols = ['Фаза сезона', 'Критерии фаз по среднесуточной']
    for col in categorical_cols:
        if col in df_processed.columns:
            le = LabelEncoder()
            # Используем fit_transform для новых данных, но это может вызвать проблемы
            # В идеале нужно сохранить и переиспользовать encoder из обучения
            df_processed[col + '_encoded'] = le.fit_transform(df_processed[col].astype(str))

    # Добавление временных признаков
    df_processed['Декада'] = (df_processed['Месяц'] - 1) * 3 + (df_processed['День'] - 1) // 10 + 1
    df_processed['День_года'] = df_processed['Дата'].dt.dayofyear

    # === Фенологические признаки ===
    df_processed = calculate_phenological_features(df_processed)

    # === Lag-признаки ===
    df_processed = add_lag_features(df_processed)

    # === Векторизация азимута ===
    df_processed = add_azimuth_vectors(df_processed)

    # === Скользящие и накопительные признаки ===
    df_processed = add_rolling_and_cumulative_features(df_processed)

    return df_processed

def load_models(target_species):
    """Загрузка обученных моделей"""
    models = {}
    for species in target_species:
        safe_name = species.replace('\n', '_').replace(' ', '_').replace('/', '_')
        model_filename = f'best_model_{safe_name}.pkl'
        if os.path.exists(model_filename):
            try:
                with open(model_filename, 'rb') as f:
                    models[species] = pickle.load(f)
                print(f"✅ Модель для {species} загружена")
            except Exception as e:
                print(f"❌ Ошибка загрузки модели для {species}: {e}")
        else:
            print(f"⚠️ Файл модели {model_filename} не найден")
    return models

def prepare_prediction_data(new_data, model_features):
    """Подготовка данных для прогнозирования"""
    # Создаем копию данных
    prediction_data = new_data.copy()
    
    # Добавляем отсутствующие признаки и заполняем нулями
    for feature in model_features:
        if feature not in prediction_data.columns:
            prediction_data[feature] = 0
            print(f"⚠️ Добавлен отсутствующий признак: {feature}")
    
    # Убеждаемся, что порядок признаков совпадает с порядком при обучении
    prediction_data = prediction_data[model_features]
    
    # Заполняем пропущенные значения
    prediction_data = prediction_data.fillna(0)
    
    return prediction_data

def make_predictions(new_data, models, target_species):
    """Создание прогнозов для всех видов"""
    predictions = {}
    
    for species in target_species:
        if species not in models:
            print(f"⚠️ Модель для {species} не найдена")
            continue
            
        model_data = models[species]
        
        try:
            # Подготавливаем данные для этого вида
            X_pred = prepare_prediction_data(new_data, model_data['features'])
            
            # Масштабируем признаки
            X_pred_scaled = model_data['scaler'].transform(X_pred)
            
            # Делаем прогноз
            y_pred = model_data['model'].predict(X_pred_scaled)
            
            # Сохраняем прогноз
            predictions[species] = y_pred
            
            print(f"✅ Прогноз для {species}: {len(y_pred)} значений")
            
        except Exception as e:
            print(f"❌ Ошибка прогноза для {species}: {e}")
    
    return predictions

def create_prediction_report(new_data, predictions, target_species):
    """Создание отчета с прогнозами"""
    # Создаем копию исходных данных
    report_df = new_data[['Год', 'Месяц', 'День']].copy()
    
    if 'Дата' in new_data.columns:
        report_df['Дата'] = new_data['Дата']
    
    # Добавляем прогнозы для каждого вида
    for species in target_species:
        if species in predictions:
            report_df[species + '_прогноз'] = predictions[species]
    
    # Добавляем суммарный прогноз
    forecast_columns = [col for col in report_df.columns if '_прогноз' in col]
    if forecast_columns:
        report_df['Общий_прогноз_пыльцы'] = report_df[forecast_columns].sum(axis=1)
    
    return report_df

def visualize_predictions(report_df, target_species):
    """Визуализация прогнозов"""
    os.makedirs('прогнозы_визуализация', exist_ok=True)
    
    # Временные ряды прогнозов для каждого вида
    forecast_columns = [col for col in report_df.columns if '_прогноз' in col and 'Общий' not in col]
    
    if forecast_columns and 'Дата' in report_df.columns:
        plt.figure(figsize=(15, 10))
        
        for i, species_col in enumerate(forecast_columns):
            plt.subplot(4, 3, i+1)
            species_name = species_col.replace('_прогноз', '')
            plt.plot(report_df['Дата'], report_df[species_col], linewidth=2)
            plt.title(f'Прогноз {species_name}')
            plt.xlabel('Дата')
            plt.ylabel('Концентрация')
            plt.xticks(rotation=45)
            plt.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.savefig('прогнозы_визуализация/прогнозы_по_видам.png', dpi=300, bbox_inches='tight')
        plt.close()
    
    # Общий прогноз
    if 'Общий_прогноз_пыльцы' in report_df.columns and 'Дата' in report_df.columns:
        plt.figure(figsize=(12, 6))
        plt.plot(report_df['Дата'], report_df['Общий_прогноз_пыльцы'], linewidth=2, color='red')
        plt.title('Общий прогноз концентрации пыльцы')
        plt.xlabel('Дата')
        plt.ylabel('Суммарная концентрация')
        plt.xticks(rotation=45)
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.savefig('прогнозы_визуализация/общий_прогноз.png', dpi=300, bbox_inches='tight')
        plt.close()
    
    # Тепловая карта по месяцам и видам
    if 'Месяц' in report_df.columns and forecast_columns:
        monthly_predictions = report_df.groupby('Месяц')[forecast_columns].mean()
        
        plt.figure(figsize=(12, 8))
        sns.heatmap(monthly_predictions.T, annot=True, fmt='.1f', cmap='YlOrRd')
        plt.title('Среднемесячные прогнозы концентрации пыльцы')
        plt.xlabel('Месяц')
        plt.ylabel('Вид пыльцы')
        plt.tight_layout()
        plt.savefig('прогнозы_визуализация/тепловая_карта_прогнозов.png', dpi=300, bbox_inches='tight')
        plt.close()

# === ОСНОВНОЙ БЛОК ПРОГНОЗИРОВАНИЯ ===
if __name__ == "__main__":
    # Целевые виды (должны совпадать с обученными моделями)
    target_species = [
        'Salix Ива',
        'Betula Береза',
        ' Populus Тополь',
        'Acer Клён',
        ' Picea Ель',
        'Сосна Pinus',
        'Poaceae\nЗлаки',
        'Plantago\nПодорожник',
        'Urtica\nКрапива',
        'Artemisia\nПолынь',
        'Chenopodioideae\nМаревые'
    ]
    
    print("🎯 ЗАПУСК ПРОГНОЗИРОВАНИЯ")
    print("=" * 50)
    
    # 1. Загружаем данные для прогноза
    print("📁 Загружаем данные для прогноза...")
    try:
        # Загружаем данные из файла проверки
        prediction_file_path = r"E:\Python_evening\Пыльца\Проверка_прогноза.xlsx"
        new_data = pd.read_excel(prediction_file_path)
        print(f"✅ Данные загружены: {new_data.shape}")
        
        # Проверяем наличие обязательных столбцов
        required_columns = ['Год', 'Месяц', 'День', 'Среднесуточная температура', 'Осадки']
        missing_columns = [col for col in required_columns if col not in new_data.columns]
        if missing_columns:
            print(f"⚠️ Отсутствуют обязательные столбцы: {missing_columns}")
        
    except Exception as e:
        print(f"❌ Ошибка загрузки данных: {e}")
        exit()
    
    # 2. Применяем предобработку
    print("🔧 Применяем предобработку признаков...")
    try:
        new_data_processed = preprocess_features(new_data)
        print(f"✅ Данные обработаны: {new_data_processed.shape}")
        
        # Сохраняем обработанные данные
        new_data_processed.to_excel('данные_для_прогноза_обработанные.xlsx', index=False)
        print("💾 Обработанные данные сохранены в 'данные_для_прогноза_обработанные.xlsx'")
        
    except Exception as e:
        print(f"❌ Ошибка предобработки: {e}")
        exit()
    
    # 3. Загружаем обученные модели
    print("🤖 Загружаем обученные модели...")
    models = load_models(target_species)
    
    if not models:
        print("❌ Не удалось загрузить ни одну модель!")
        exit()
    
    # 4. Создаем прогнозы
    print("📊 Создаем прогнозы...")
    predictions = make_predictions(new_data_processed, models, target_species)
    
    if not predictions:
        print("❌ Не удалось создать ни одного прогноза!")
        exit()
    
    # 5. Создаем отчет
    print("📈 Формируем отчет...")
    prediction_report = create_prediction_report(new_data_processed, predictions, target_species)
    
    # Сохраняем отчет
    prediction_report.to_excel('прогноз_пыльцы_результаты.xlsx', index=False)
    print("💾 Отчет с прогнозами сохранен в 'прогноз_пыльцы_результаты.xlsx'")
    
    # 6. Создаем визуализации
    print("🎨 Создаем визуализации...")
    visualize_predictions(prediction_report, target_species)
    print("💾 Визуализации сохранены в папку 'прогнозы_визуализация'")
    
    # 7. Сводная статистика
    print("\n📋 СВОДКА ПРОГНОЗОВ:")
    print("=" * 50)
    
    forecast_columns = [col for col in prediction_report.columns if '_прогноз' in col and 'Общий' not in col]
    for col in forecast_columns:
        species_name = col.replace('_прогноз', '')
        values = prediction_report[col]
        print(f"🌿 {species_name:25} | Мин: {values.min():6.1f} | Макс: {values.max():6.1f} | Сред: {values.mean():6.1f}")
    
    if 'Общий_прогноз_пыльцы' in prediction_report.columns:
        total = prediction_report['Общий_прогноз_пыльцы']
        print(f"📊 ОБЩИЙ ПРОГНОЗ         | Мин: {total.min():6.1f} | Макс: {total.max():6.1f} | Сред: {total.mean():6.1f}")
    
    print("\n✅ ПРОГНОЗИРОВАНИЕ ЗАВЕРШЕНО!")
    print("📁 Результаты:")
    print("   - прогноз_пыльцы_результаты.xlsx - таблица с прогнозами")
    print("   - данные_для_прогноза_обработанные.xlsx - обработанные входные данные")
    print("   - папка 'прогнозы_визуализация' - графики и визуализации")

🎯 ЗАПУСК ПРОГНОЗИРОВАНИЯ
📁 Загружаем данные для прогноза...
✅ Данные загружены: (214, 15)
🔧 Применяем предобработку признаков...
✅ Данные обработаны: (214, 67)
💾 Обработанные данные сохранены в 'данные_для_прогноза_обработанные.xlsx'
🤖 Загружаем обученные модели...
✅ Модель для Salix Ива загружена
✅ Модель для Betula Береза загружена
✅ Модель для  Populus Тополь загружена
✅ Модель для Acer Клён загружена
✅ Модель для  Picea Ель загружена
✅ Модель для Сосна Pinus загружена
✅ Модель для Poaceae
Злаки загружена
✅ Модель для Plantago
Подорожник загружена
✅ Модель для Urtica
Крапива загружена
✅ Модель для Artemisia
Полынь загружена
✅ Модель для Chenopodioideae
Маревые загружена
📊 Создаем прогнозы...
⚠️ Добавлен отсутствующий признак: Среднесуточная температура_фаза_mean
⚠️ Добавлен отсутствующий признак: Среднесуточная температура_фаза_max
⚠️ Добавлен отсутствующий признак: Среднесуточная температура_фаза_min
⚠️ Добавлен отсутствующий признак: Среднесуточная температура_фаза_median
⚠️ Добав